In [21]:
library(tidyverse)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/



In [18]:
red<- read_csv2("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv") 

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (5): volatile acidity, citric acid, chlorides, density, sulphates
dbl (2): total sulfur dioxide, quality

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [19]:
red_desc <- red |> arrange(desc(quality))
red_desc

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
79,0.35,0.46,36,0.078,15,37,0.9973,335,0.86,128,8
103,0.32,0.45,64,0.073,5,13,0.9976,323,0.82,126,8
56,0.85,0.05,14,0.045,12,88,0.9924,356,0.82,129,8
126,0.31,0.72,22,0.072,6,29,0.9987,288,0.82,98,8
113,0.62,0.67,52,0.086,6,19,0.9988,322,0.69,134,8
94,0.3,0.56,28,0.08,6,17,0.9964,315,0.92,117,8
107,0.35,0.53,26,0.07,5,16,0.9972,315,0.65,11,8
107,0.35,0.53,26,0.07,5,16,0.9972,315,0.65,11,8
5,0.42,0.24,2,0.06,19,50,0.9917,372,0.74,14,8


In [25]:
red_split<-initial_split(red, prop=0.75, strata=quality)
red_train<-training(red_split)
red_test<-testing(red_split)
red_test

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
112,0.28,0.56,19,0.075,17,60,0.998,316,0.58,98,6
79,0.6,0.06,16,0.069,15,59,0.9964,33,0.46,94,5
67,0.58,0.08,18,0.097,15,65,0.9959,328,0.54,92,5
75,0.5,0.36,61,0.071,17,102,0.9978,335,0.8,105,5
89,0.62,0.18,38,0.176,52,145,0.9986,316,0.88,92,5
85,0.28,0.56,18,0.092,35,103,0.9969,33,0.75,105,7
74,0.59,0.08,44,0.086,6,29,0.9974,338,0.5,9,4
76,0.39,0.31,23,0.082,23,71,0.9982,352,0.65,97,5
79,0.43,0.21,16,0.106,10,37,0.9966,317,0.91,95,5
